<a href="https://colab.research.google.com/github/eunhabaek/Dacon/blob/main/%EC%A0%9C1%ED%9A%8C_%EC%8B%A0%EC%95%BD%EA%B0%9C%EB%B0%9C_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 제1회 신약개발 AI 경진대회

## 데이터 탐색

In [ ]:
import pandas as pd
import numpy as np

#데이터 가져오기
train=pd.read_csv('../data/train.csv',index_col='id')
test=pd.read_csv('../data/test.csv',index_col='id')
submission=pd.read_csv('../data/sample_submission.csv',index_col='id')

#데이터 컬럼명 확인
print(train.columns)
print(test.columns)
print(submission.columns)

#데이터 확인
print(train.head())
#print(test.head())
print(submission.head())

#데이터 정보 요약 함수 만들기
def summaryTable(df):
    print('데이터 구조: ',df.shape)
    summary=pd.DataFrame(df.dtypes,columns=['데이터 타입'])
    summary=summary.reset_index()
    summary=summary.rename(columns={'index':'피쳐'})
    summary['NAN 개수']=df.isnull().sum().values
    summary['데이터 종류']=df.nunique().values
    return summary

#summaryTable(test)
#summaryTable(train)


FileNotFoundError: ignored

In [ ]:
import sys
sys.cwd


## 결측치 처리

In [ ]:
import statistics as st

#전처리 및 데이터 탐색을 위해서 train+test data 합치기
target=['MLM','HLM']

#target 열 추출
y=train[target]

train=train.drop(target,axis=1)

#all data 생성
all_data=pd.concat([train,test])

#문자열 데이터 제외
all_data=all_data.drop('SMILES',axis=1)

#분석에 사용 할 피쳐
print(all_data)

#train+test AlogP 데이터의 중앙 값 확인
alogp_med=st.median(all_data.AlogP)

print(alogp_med)

#결측치를 중앙 값으로 결측치 처리
all_data['AlogP'].fillna(alogp_med,inplace=True)

summaryTable(all_data)

## 다중 공선성 확인

In [ ]:
## 피쳐간 상관도 확인
all_data.corr()

#AlogP와 LogD 상관성 확인
#Num_H_Acceptors과 Molecular_PolarSurfaceArea 상관성 확인

#VIF 값 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor


vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(all_data.values, i) for i in range(all_data.shape[1])]
vif["features"] = all_data.columns
print(vif.round(1))


In [ ]:
# VIF 값 높은 열 빼고 다시 확인

X=all_data.drop(['AlogP','Molecular_Weight','Molecular_PolarSurfaceArea'],axis=1)

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif.round(1))

#train, test set 다시 나누기
n_train=len(train)

X_train=all_data[:n_train]
X_test=all_data[n_train:]

## 모델 생성 및 학습

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# degree는 차수
# degree 높아질수록 곡선이 복잡해지며. 정확성 높이나 overfitting 가능성 커짐
polynorm_svm=Pipeline([
    ('poly_features',PolynomialFeatures(degree=5)),
    ('scaler',StandardScaler()),
    ('linear_svc',LinearSVC(C=1,loss='hinge',random_state=42))
])
#print(y.MLM)
#polynorm_svm.fit(X_train,y.MLM)

In [ ]:
from sklearn.linear_model import LinearRegression

lr_mlm=LinearRegression()
lr_hlm=LinearRegression()

#모델 훈련
lr_mlm.fit(X_train,y.MLM)
lr_hlm.fit(X_train,y.HLM)

submission['MLM']=lr_mlm.predict(X_test)
submission['HLM']=lr_hlm.predict(X_test)

In [ ]:
submission.to_csv('../data/sample_submission2.csv')